In [1]:
import pandas as pd 
import os 

In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("preprocessing").getOrCreate()

23/02/28 21:16:34 WARN Utils: Your hostname, imhaneul-ui-MacBookPro.local resolves to a loopback address: 127.0.0.1; using 172.30.1.74 instead (on interface en0)
23/02/28 21:16:34 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/02/28 21:16:35 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
directory = f"{os.getcwd()}/data"
print(directory)
sg = "gyoung.csv"
bus = "bus.csv"

sg_data = spark.read.csv(f"{directory}/{sg}", inferSchema=True, header=True)
bus_data = spark.read.csv(f"{directory}/{bus}", inferSchema=True, header=True)

sg_data = sg_data.createOrReplaceTempView("sg_data")
bus_data = bus_data.createOrReplaceTempView("bus_data")

/Users/imhaneul/Documents/project/BusStopPlace/SparkPreprocessing/analysis/data


In [12]:
# """
# /Users/imhaneul/Documents/project/BusStopPlace/SparkPreprocessing/analysis/data
# root
#  |-- 정류장아이디: string (nullable = true)
#  |-- 정류장 명칭: string (nullable = true)
#  |-- 정류장 유형: double (nullable = true)
#  |-- 중앙차로 여부: string (nullable = true)
#  |-- 노드영문명: string (nullable = true)
#  |-- 위도: double (nullable = true)
#  |-- 경도: double (nullable = true)
#  |-- 수집일시: string (nullable = true)
#  |-- 단축아이디: double (nullable = true)
#  |-- 도시코드: integer (nullable = true)
#  |-- 도시명: string (nullable = true)
#  |-- _c0: string (nullable = true)
#     bus_data.`정류장아이디` as id,
#     bus_data.`정류장 명칭`, as name,
#     bus_data._c0 as location
#"""
qs = """
SELECT 
    `정류장아이디` as id,
    sg_data.`도시명` as city,
    `정류장 명칭` as name,
    `위도` as gps_lati,
    `경도` as gps_long,
    `수집일시` as collected_at
FROM 
    bus_data
    JOIN
        sg_data
    ON
        bus_data.`도시명` = sg_data.`도시명`
"""
sg = spark.sql(qs).toPandas()

,id,city,name,gps_lati,gps_long,collected_at
0,ICB163000077,인천광역시,학익사거리,37.442734,126.663838,2021/09/16 04:01:02
1,ICB163000078,인천광역시,법원검찰청입구,37.441503,126.668130,2021/09/16 04:01:02
2,ICB163000079,인천광역시,한양아파트,37.450492,126.638504,2021/09/16 04:01:02
3,ICB163000080,인천광역시,학익시장(법원검찰청),37.442449,126.665103,2021/09/16 04:01:02
4,ICB163000081,인천광역시,gym80fitness,37.451758,126.634821,2021/09/16 04:01:02
...,...,...,...,...,...,...
55513,GGB277103271,성남시,성남IC출입(경유),37.427200,127.125850,2021/09/16 04:30:05
55514,GGB277103272,성남시,성남TG(경유),37.436533,127.122183,2021/09/16 04:30:05
55515,GGB277103273,성남시,성남TG(경유),37.435083,127.123850,2021/09/16 04:30:04
55516,GGB277103274,하남시,하남JC(경유),37.531067,127.195050,2021/09/16 04:30:05


In [5]:
sg.to_csv("test.csv", index=False, index_label=False)